# 作業重點:

(1)以, Adam, 為例, 調整 batch_size, epoch , 觀察accurancy, loss 的變化

(2)以同一模型, 分別驗證 SGD, Adam, Rmsprop 的 accurancy

# 作業目標:
    
    取得各種優化器的運算結果

In [1]:
from __future__ import print_function
import keras
import tensorflow
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import optimizers
import itertools
import pandas as pd 


In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
LEARNING_RATE = 1e-3
num_classes = 10
data_augmentation = True
num_predictions = 20


In [4]:

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# 資料正規化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [6]:
#    第一步：選擇模型, 順序模型是多個網絡層的線性堆疊
 
model = Sequential()

#   第二步：構建網絡層
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense( 10)) # 輸出結果是10個類別，所以維度是10   
model.add(Activation('softmax')) # 最後一層用softmax作為激活函數

In [7]:
# 模型建立完成後，統計參數總量
print("Total Parameters：%d" % model.count_params())

Total Parameters：1250858


In [8]:
# 輸出模型摘要資訊
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                        

### (1)以, Adam, 為例, 調整 batch_size, epoch , 觀察accurancy, loss 的變化

### (2)以同一模型, 分別驗證 SGD, Adam, Rmsprop 的 accurancy

In [9]:
batch_sizes = [512]
epochs = [20, 30]
optimizer_set = [keras.optimizers.gradient_descent_v2.SGD(learning_rate=LEARNING_RATE, nesterov=True, momentum=0.95),
                 tensorflow.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                 tensorflow.keras.optimizers.RMSprop(learning_rate=LEARNING_RATE)]

results = []
for i, (batch_size, epoch, opt) in enumerate(itertools.product(batch_sizes, epochs, optimizer_set)):
    print(f"exp: {i}, batch_size={batch_size}, epochs: {epochs}, optimizer:{type(opt).__name__}")
    

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

        
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    history=model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epoch,
              validation_data=(x_test, y_test),
              shuffle=True)       
    
    results.append( {'exp': i,
                     'batch_size': batch_size,
                     'epoch': epoch,
                     'optimizer': type(opt).__name__,
                     'train-loss': model.history.history["loss"],
                     'valid-loss': model.history.history["val_loss"],
                     'train-acc': model.history.history["accuracy"],
                     'valid-acc': model.history.history["val_accuracy"],
                    })    
        

experiments = pd.DataFrame(results, columns =['exp', 'batch_size', 'epoch', 'optimizer', 'train-loss', 'valid-loss', 'train-acc', 'valid-acc'])
experiments

exp: 0, batch_size=512, epochs: [20, 30], optimizer:SGD
Epoch 1/20
98/98 [==============================] - 7s 30ms/step - loss: 2.2978 - accuracy: 0.1158 - val_loss: 2.2827 - val_accuracy: 0.1452
Epoch 2/20
98/98 [==============================] - 2s 22ms/step - loss: 2.2498 - accuracy: 0.1518 - val_loss: 2.1821 - val_accuracy: 0.2175
Epoch 3/20
98/98 [==============================] - 2s 22ms/step - loss: 2.1242 - accuracy: 0.2123 - val_loss: 2.0260 - val_accuracy: 0.2674
Epoch 4/20
98/98 [==============================] - 2s 22ms/step - loss: 2.0395 - accuracy: 0.2432 - val_loss: 1.9793 - val_accuracy: 0.2822
Epoch 5/20
98/98 [==============================] - 2s 22ms/step - loss: 1.9984 - accuracy: 0.2638 - val_loss: 1.9387 - val_accuracy: 0.3024
Epoch 6/20
98/98 [==============================] - 2s 22ms/step - loss: 1.9627 - accuracy: 0.2830 - val_loss: 1.8907 - val_accuracy: 0.3259
Epoch 7/20
98/98 [==============================] - 2s 22ms/step - loss: 1.9187 - accuracy: 0.3018

Epoch 18/20
98/98 [==============================] - 2s 25ms/step - loss: 0.3771 - accuracy: 0.8673 - val_loss: 0.6815 - val_accuracy: 0.7925
Epoch 19/20
98/98 [==============================] - 2s 25ms/step - loss: 0.3588 - accuracy: 0.8712 - val_loss: 0.6722 - val_accuracy: 0.7925
Epoch 20/20
98/98 [==============================] - 2s 25ms/step - loss: 0.3560 - accuracy: 0.8731 - val_loss: 0.7027 - val_accuracy: 0.7855
exp: 3, batch_size=512, epochs: [20, 30], optimizer:SGD
Epoch 1/30
98/98 [==============================] - 3s 25ms/step - loss: 0.2822 - accuracy: 0.8991 - val_loss: 0.6463 - val_accuracy: 0.8027
Epoch 2/30
98/98 [==============================] - 2s 24ms/step - loss: 0.2693 - accuracy: 0.9046 - val_loss: 0.6470 - val_accuracy: 0.8040
Epoch 3/30
98/98 [==============================] - 2s 23ms/step - loss: 0.2615 - accuracy: 0.9060 - val_loss: 0.6514 - val_accuracy: 0.8024
Epoch 4/30
98/98 [==============================] - 2s 23ms/step - loss: 0.2610 - accuracy: 0.9

Epoch 25/30
98/98 [==============================] - 2s 25ms/step - loss: 0.1958 - accuracy: 0.9295 - val_loss: 0.7115 - val_accuracy: 0.8048
Epoch 26/30
98/98 [==============================] - 2s 23ms/step - loss: 0.2042 - accuracy: 0.9276 - val_loss: 0.6923 - val_accuracy: 0.8061
Epoch 27/30
98/98 [==============================] - 2s 23ms/step - loss: 0.1954 - accuracy: 0.9301 - val_loss: 0.7154 - val_accuracy: 0.8068
Epoch 28/30
98/98 [==============================] - 2s 23ms/step - loss: 0.1958 - accuracy: 0.9308 - val_loss: 0.6965 - val_accuracy: 0.8018
Epoch 29/30
98/98 [==============================] - 2s 23ms/step - loss: 0.2003 - accuracy: 0.9290 - val_loss: 0.7136 - val_accuracy: 0.8017
Epoch 30/30
98/98 [==============================] - 2s 23ms/step - loss: 0.1888 - accuracy: 0.9323 - val_loss: 0.7210 - val_accuracy: 0.8029
exp: 5, batch_size=512, epochs: [20, 30], optimizer:RMSprop
Epoch 1/30
98/98 [==============================] - 3s 27ms/step - loss: 0.2336 - accura

,exp,batch_size,epoch,optimizer,train-loss,valid-loss,train-acc,valid-acc
0,0,512,20,SGD,"[2.297805070877075, 2.249804973602295, 2.12421...","[2.2827370166778564, 2.182128667831421, 2.0260...","[0.11584000289440155, 0.15177999436855316, 0.2...","[0.1451999992132187, 0.2175000011920929, 0.267..."
1,1,512,20,Adam,"[1.5388472080230713, 1.2835760116577148, 1.172...","[1.2855613231658936, 1.1459367275238037, 1.067...","[0.4539799988269806, 0.5415800213813782, 0.585...","[0.546999990940094, 0.5952000021934509, 0.6243..."
2,2,512,20,RMSprop,"[0.7913756370544434, 0.5913901329040527, 0.569...","[0.6732553243637085, 0.7440592050552368, 0.720...","[0.7649199962615967, 0.7927799820899963, 0.799...","[0.7688999772071838, 0.7523999810218811, 0.762..."
3,3,512,30,SGD,"[0.2821560800075531, 0.26928815245628357, 0.26...","[0.6462984681129456, 0.6469695568084717, 0.651...","[0.8991199731826782, 0.9046199917793274, 0.905...","[0.8026999831199646, 0.8040000200271606, 0.802..."
4,4,512,30,Adam,"[0.2882141172885895, 0.2889169752597809, 0.286...","[0.6524515151977539, 0.6515591740608215, 0.640...","[0.8961600065231323, 0.896619975566864, 0.8967...","[0.800599992275238, 0.7998999953269958, 0.7996..."
5,5,512,30,RMSprop,"[0.23363080620765686, 0.2209729701280594, 0.22...","[0.7401626110076904, 0.7642381191253662, 0.695...","[0.9188399910926819, 0.9233800172805786, 0.921...","[0.7964000105857849, 0.7993999719619751, 0.804..."


## load model

In [ ]:
from keras.models import load_model
columns = ['loss_score', 'accuracy_score', 'optimizer', 'loss_function', 'epoch', 'learning_rate', 'decacy', \
           'momentum']
loss_accuracy_sore = []

In [ ]:
model_adm1 = load_model(save_dir+"/keras_cifar10_opt_adm_epochs_20.h5")
scores_adm1 = model.evaluate(x_test,y_test,batch_size=200,verbose= 0)
loss_accuracy_sore.append(['', scores_adm1, 'adm', 'categorical_crossentropy', 20, '', '', ''])

In [ ]:
model_RMSprop = load_model(save_dir+"/keras_cifar10_opt_RMSprop_epochs_30_lr_0.01-v1r01.h5")
scores_RMSprop = model.evaluate(x_test,y_test,batch_size=200,verbose= 0)
loss_accuracy_sore.append(['', scores_RMSprop, 'RMSprop', 'categorical_crossentropy', 30, '', '', ''])

In [ ]:
loss_accuracy_sore

In [ ]:
#    第六步：輸出
import numpy 

print ( " test set " )
scores = model.evaluate(x_test,y_test,batch_size=200,verbose= 0)
print ( "" )
#print ( " The test loss is %f " % scores)
print ( " The test loss is %f ", scores)


result = model.predict(x_test,batch_size=200,verbose= 0)

result_max = numpy.argmax(result, axis = 1 )
test_max = numpy.argmax(y_test, axis = 1 )

result_bool = numpy.equal(result_max, test_max)
true_num = numpy.sum(result_bool)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valiidation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valiidation'], loc='upper left')
plt.show()